<a href="https://colab.research.google.com/github/deathboydmi/DynamicChannelPruning/blob/master/dp_squeezenet_train_val.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python3 --version

Python 3.6.7


In [2]:
!pip3 install torch==1.1.0

In [3]:
!pip3 uninstall torchvision -y
!git clone https://github.com/pytorch/vision.git

Uninstalling torchvision-0.2.3a0+d534785:
  Successfully uninstalled torchvision-0.2.3a0+d534785
fatal: destination path 'vision' already exists and is not an empty directory.


In [4]:
!ls
%cd vision
!python setup.py install
%cd ../

data  sample_data  vision
/content/vision
running install
running bdist_egg
running egg_info
writing torchvision.egg-info/PKG-INFO
writing dependency_links to torchvision.egg-info/dependency_links.txt
writing requirements to torchvision.egg-info/requires.txt
writing top-level names to torchvision.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
writing manifest file 'torchvision.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
copying torchvision/version.py -> build/lib/torchvision
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/torchvision
creating build/bdist.linux-x86_64/egg/torchvision/models
copying build/lib/torchvision/models/mobilenet.py -> build/bdist.linux-x86_64/egg/torchvision/models
copying build/lib/torchvision/models/densenet.py -> build/bdist.linux-x86_64/egg/torchvision/models
copying build/lib/torchvision/models/squeezenet.py -> build/bdist.linux-x86_64/egg/torc

In [5]:
%ll


total 12
drwxr-xr-x  3 root 4096 May  5 11:41 data/
drwxr-xr-x  1 root 4096 Apr 29 16:32 sample_data/
drwxr-xr-x 10 root 4096 May  5 06:44 vision/


In [6]:
%ll
%cd data
!wget http://www.image-net.org/challenges/LSVRC/2012/nnoupb/ILSVRC2012_img_train.tar -c
!wget http://www.image-net.org/challenges/LSVRC/2012/nnoupb/ILSVRC2012_img_val.tar -c
!wget http://www.image-net.org/challenges/LSVRC/2012/nnoupb/ILSVRC2012_devkit_t12.tar.gz -c
%cd ..

total 12
drwxr-xr-x  3 root 4096 May  5 11:41 data/
drwxr-xr-x  1 root 4096 Apr 29 16:32 sample_data/
drwxr-xr-x 10 root 4096 May  5 06:44 vision/
/content/data
--2019-05-05 12:34:26--  http://www.image-net.org/challenges/LSVRC/2012/nnoupb/ILSVRC2012_img_train.tar
Resolving www.image-net.org (www.image-net.org)... 171.64.68.16
Connecting to www.image-net.org (www.image-net.org)|171.64.68.16|:80... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

--2019-05-05 12:34:27--  http://www.image-net.org/challenges/LSVRC/2012/nnoupb/ILSVRC2012_img_val.tar
Resolving www.image-net.org (www.image-net.org)... 171.64.68.16
Connecting to www.image-net.org (www.image-net.org)|171.64.68.16|:80... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

--2019-05-05 12:34:29--  http://www.image-net.org/challenges/LSVRC/2012

In [0]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.nn.init as init


class DynamicPruning(nn.Module):
    def __init__(self, in_channels, hidden_layer_channels=None):
        super(DynamicPruning, self).__init__()

        if hidden_layer_channels is None:
            hidden_layer_channels = in_channels // 16
            if hidden_layer_channels < 4:
                hidden_layer_channels = 4

        self.gavgpool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Conv2d(in_channels, hidden_layer_channels,
                             kernel_size=1, stride=1)
        self.fc2 = nn.Conv2d(hidden_layer_channels,
                             in_channels, kernel_size=1, stride=1)

        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.constant_(self.fc1.bias, 0)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.constant_(self.fc2.bias, 0)

    def forward(self, x):

        x = self.gavgpool(x)

        x = self.fc1(x)
        x = torch.clamp(x, 0, 1)

        x = self.fc2(x)
        x = torch.clamp(x, 0, 1)

        return x


class DP_Conv2d(nn.Module):
    def __init__(self, in_channels=0, out_channels=0,
                 kernel_size=0, stride=1, padding=0, dilation=1,
                 groups=1, bias=True, padding_mode='zeros',
                 conv2d=None, hidden_layer_channels=None):
        super(DP_Conv2d, self).__init__()
        if in_channels is 0 or out_channels is 0 or kernel_size is 0:
            if conv2d is None:
                assert()
            else:
                self.__init__from_Conv2d(conv2d, hidden_layer_channels)
        else:
            if hidden_layer_channels is None:
                hidden_layer_channels = out_channels // 16
                if hidden_layer_channels < 4:
                    hidden_layer_channels = 4

            self.prun = DynamicPruning(in_channels, hidden_layer_channels)
            self.conv = nn.Conv2d(in_channels, out_channels,
                                  kernel_size, stride=stride,
                                  padding=padding, dilation=dilation,
                                  groups=groups, bias=bias,
                                  padding_mode=padding_mode)

    def __init__from_Conv2d(self, conv2d, hidden_layer_channels=None):
        if not isinstance(conv2d, nn.Conv2d):
            assert()
        if hidden_layer_channels is None:
            hidden_layer_channels = conv2d.out_channels // 16
            if hidden_layer_channels < 4:
                hidden_layer_channels = 4

        self.prun = DynamicPruning(
            conv2d.in_channels, hidden_layer_channels)
        self.conv = conv2d

    def forward(self, x):
        dp_res = self.prun(x)
        x = x * dp_res
        x = self.conv(x)

        return x
    


In [8]:
import torchvision.models as models


class FireDP(nn.Module):
    def __init__(self, fire):
        super(FireDP, self).__init__()

        self.squeeze = fire.squeeze
        self.squeeze_activation = fire.squeeze_activation

        self.expand1x1 = fire.expand1x1
        self.expand1x1_activation = fire.expand1x1_activation

        self.prun = DynamicPruning(self.squeeze.out_channels)

        self.expand3x3 = fire.expand3x3
        self.expand3x3_activation = fire.expand3x3_activation

    def forward(self, x):
        x = self.squeeze_activation(self.squeeze(x))
        dp_res = self.prun(x)
        dp_res = dp_res * x

        return torch.cat([
            self.expand1x1_activation(self.expand1x1(x)),
            self.expand3x3_activation(self.expand3x3(dp_res))
        ], 1)


class SqueezeNet_DP(nn.Module):
    def __init__(self, version=1.0, num_classes=1000):
        super(SqueezeNet_DP, self).__init__()

        self.num_classes = num_classes

        pretrain_model = models.squeezenet1_1(
            pretrained=True, num_classes=num_classes)

        self.features = pretrain_model.features
        self.features[3] = FireDP(fire=self.features[3])
        self.features[4] = FireDP(fire=self.features[4])
        self.features[6] = FireDP(fire=self.features[6])
        self.features[7] = FireDP(fire=self.features[7])

        self.classifier = pretrain_model.classifier

    def forward(self, x):

        x = self.features(x)
        x = self.classifier(x)

        return x.view(x.size(0), self.num_classes)

print(SqueezeNet_DP())

SqueezeNet_DP(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): FireDP(
      (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace)
      (prun): DynamicPruning(
        (gavgpool): AdaptiveAvgPool2d(output_size=1)
        (fc1): Conv2d(16, 4, kernel_size=(1, 1), stride=(1, 1))
        (fc2): Conv2d(4, 16, kernel_size=(1, 1), stride=(1, 1))
      )
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace)
    )
    (4): FireDP(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      

In [0]:
import random
import shutil
import time
import warnings

import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets

best_prec1 = 0
best_total_num_zeros = 0

DP_zeros = 0
DP_loss = 0

DP_num_zeros_for_each_block = [0, 0, 0, 0]

learning_rate = 0.04
batch_size = 128
epochs = 70
momentum = 0.9
weight_decay = 0.0002
resume = False
start_epoch = 0
evaluate = False
print_freq = 10


def main():
    global best_prec1, best_total_num_zeros, DP_num_zeros_for_each_block
    global learning_rate, batch_size, epochs, momentum, weight_decay
    global start_epoch, resume, evaluate

    random.seed(271728)
    torch.manual_seed(271728)
    cudnn.deterministic = True

    model = SqueezeNet_DP()

    criterion = nn.CrossEntropyLoss().cuda()
    optimizer = torch.optim.SGD(filter(
        lambda p: p.requires_grad, model.parameters()),
        learning_rate,
        momentum=momentum,
        weight_decay=weight_decay)

    if resume:
        if os.path.isfile(resume):
            print("=> loading checkpoint '{}'".format(resume))
            checkpoint = torch.load(resume)
            start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(resume, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(resume))

    cudnn.benchmark = True

    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    train_dataset = datasets.ImageNet('./data/', split='train', download=True,
                                      transform=transforms.Compose([
                                          transforms.RandomResizedCrop(224),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.ToTensor(),
                                          normalize
                                      ]))

    train_sampler = None

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size,
        shuffle=(train_sampler is None),
        num_workers=8, pin_memory=True, sampler=train_sampler)

    val_dataset = datasets.ImageNet('./data/', split='val', download=True, transform=transforms.Compose(
        [transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor(), normalize]))

    val_loader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=batch_size, shuffle=False,
                                             num_workers=8, pin_memory=True)

    if evaluate:
        validate(val_loader, model, criterion)
        return

    for epoch in range(start_epoch, epochs):
        train(train_loader, model, criterion, optimizer, epoch)
        adjust_learning_rate(optimizer, epoch)

        prec1 = validate(val_loader, model, criterion)

        is_best_acc = prec1 > best_prec1

        DP_total_num_zeros = 0
        for num_zeros in DP_num_zeros_for_each_block:
            DP_total_num_zeros += num_zeros
        DP_num_zeros_for_each_block = [0, 0, 0, 0]

        is_best_prun = DP_total_num_zeros > best_total_num_zeros

        best_prec1 = max(prec1, best_prec1)
        best_total_num_zeros = max(DP_total_num_zeros, best_total_num_zeros)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer': optimizer.state_dict(),
        }, is_best_acc, is_best_prun)


In [0]:
def train(train_loader, model, criterion, optimizer, epoch):
    global DP_loss, DP_zeros
    global print_freq

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()

    handle_1 = model.features[3].prun.register_forward_hook(DP_results_train)
    handle_2 = model.features[4].prun.register_forward_hook(DP_results_train)
    handle_3 = model.features[6].prun.register_forward_hook(DP_results_train)
    handle_4 = model.features[7].prun.register_forward_hook(DP_results_train)

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        data_time.update(time.time() - end)

        output = model(input)

        loss = criterion(output, target)
        print(loss, loss+DP_loss)
        loss += DP_loss

        losses.update(loss.item(), input.size(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses))

        DP_loss = 0
        DP_zeros = 0

    handle_1.remove()
    handle_2.remove()
    handle_3.remove()
    handle_4.remove()


In [0]:
def validate(val_loader, model, criterion):
    global DP_zeros, DP_num_zeros_for_each_block, DP_loss
    global print_freq

    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    dp_losses = AverageMeter()
    dp_zeros = AverageMeter()

    handle1 = model.features[3].prun.register_forward_hook(DP_get_num_zeros_1)
    handle2 = model.features[4].prun.register_forward_hook(DP_get_num_zeros_2)
    handle3 = model.features[6].prun.register_forward_hook(DP_get_num_zeros_3)
    handle4 = model.features[7].prun.register_forward_hook(DP_get_num_zeros_4)

    handle_1 = model.features[3].prun.register_forward_hook(DP_results)
    handle_2 = model.features[4].prun.register_forward_hook(DP_results)
    handle_3 = model.features[6].prun.register_forward_hook(DP_results)
    handle_4 = model.features[7].prun.register_forward_hook(DP_results)

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (input, target) in enumerate(val_loader):

            output = model(input)
            loss = criterion(output, target)
            loss += DP_loss

            prec1, prec5 = accuracy(output, target, topk=(1, 5))

            losses.update(loss.item(), input.size(0))
            dp_losses.update(DP_loss)
            dp_zeros.update(DP_zeros)
            top1.update(prec1[0], input.size(0))
            top5.update(prec5[0], input.size(0))

            batch_time.update(time.time() - end)
            end = time.time()

            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'DP_Loss {dp_loss.val:.4f} ({dp_loss.avg:.4f})\t'
                  'Number of zeros {num.val} ({num.avg:.3f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                      i, len(val_loader), batch_time=batch_time, loss=losses,
                      dp_loss=dp_losses, num=dp_zeros,
                      top1=top1, top5=top5))

            DP_loss = 0
            DP_zeros = 0

            if i == len(val_loader) - 1:
                last_bs = output.size(0)

        handle1.remove()
        handle2.remove()
        handle3.remove()
        handle4.remove()

        handle_1.remove()
        handle_2.remove()
        handle_3.remove()
        handle_4.remove()

        DP_percentage_zeros_for_each_block = []
        DP_percentage_zeros_for_each_block.append(
            DP_num_zeros_for_each_block[0] / (
                ((len(val_loader) - 1) * args.batch_size + last_bs) * 16
            )
        )
        DP_percentage_zeros_for_each_block.append(
            DP_num_zeros_for_each_block[1] / (
                ((len(val_loader) - 1) * args.batch_size + last_bs) * 16
            )
        )
        DP_percentage_zeros_for_each_block.append(
            DP_num_zeros_for_each_block[2] / (
                ((len(val_loader) - 1) * args.batch_size + last_bs) * 32
            )
        )
        DP_percentage_zeros_for_each_block.append(
            DP_num_zeros_for_each_block[3] / (
                ((len(val_loader) - 1) * args.batch_size + last_bs) * 32
            )
        )

        print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))
        print('\t', DP_percentage_zeros_for_each_block)

    return top1.avg


In [0]:
def DP_results(self, input, output):
    global DP_zeros, DP_loss
    vec = output
    vec = vec.cpu()

    for i in vec.view(-1):
        if i == 0:
            DP_zeros += 1

    DP_loss += torch.dist(vec.mean(), torch.tensor(0.5))


def DP_results_train(self, input, output):
    global DP_loss
    vec = output
    vec = vec.cpu()

    DP_loss += torch.dist(vec.mean(), torch.tensor(0.5))


def DP_get_num_zeros_1(self, input, output):
    global DP_num_zeros_for_each_block
    vec = output
    vec = vec.cpu()
    for i in vec.view(-1):
        if i == 0:
            DP_num_zeros_for_each_block[0] += 1


def DP_get_num_zeros_2(self, input, output):
    global DP_num_zeros_for_each_block
    vec = output
    vec = vec.cpu()
    for i in vec.view(-1):
        if i == 0:
            DP_num_zeros_for_each_block[1] += 1


def DP_get_num_zeros_3(self, input, output):
    global DP_num_zeros_for_each_block
    vec = output
    vec = vec.cpu()
    for i in vec.view(-1):
        if i == 0:
            DP_num_zeros_for_each_block[2] += 1


def DP_get_num_zeros_4(self, input, output):
    global DP_num_zeros_for_each_block
    vec = output
    vec = vec.cpu()
    for i in vec.view(-1):
        if i == 0:
            DP_num_zeros_for_each_block[3] += 1


In [0]:
def save_checkpoint(state, is_best_acc, is_best_prun,
                    filename='../DP_SqueezeNet/checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best_acc:
        shutil.copyfile(filename,
                        '../DP_SqueezeNet/model_best_acc.pth.tar')
    if is_best_prun:
        shutil.copyfile(filename,
                        '../DP_SqueezeNet/model_best_prun.pth.tar')


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 15 epochs"""
    lr = args.lr * (0.1 ** (epoch // 15))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res


In [0]:
main()

You set download=True, but a folder 'train' already exist in the root directory. If you want to re-download or re-extract the archive, delete the folder.
tensor(8.1584, grad_fn=<NllLossBackward>) tensor(9.4732, grad_fn=<AddBackward0>)
Epoch: [0][0/8271]	Time 33.961 (33.961)	Data 20.083 (20.083)	Loss 9.4732 (9.4732)	
tensor(6.8892, grad_fn=<NllLossBackward>) tensor(8.2524, grad_fn=<AddBackward0>)
tensor(6.9285, grad_fn=<NllLossBackward>) tensor(8.2385, grad_fn=<AddBackward0>)
tensor(7.0190, grad_fn=<NllLossBackward>) tensor(8.2817, grad_fn=<AddBackward0>)
tensor(7.1149, grad_fn=<NllLossBackward>) tensor(8.3144, grad_fn=<AddBackward0>)
tensor(7.0072, grad_fn=<NllLossBackward>) tensor(8.1494, grad_fn=<AddBackward0>)
tensor(6.9196, grad_fn=<NllLossBackward>) tensor(8.0046, grad_fn=<AddBackward0>)
tensor(6.8987, grad_fn=<NllLossBackward>) tensor(7.9288, grad_fn=<AddBackward0>)
tensor(6.9389, grad_fn=<NllLossBackward>) tensor(7.8850, grad_fn=<AddBackward0>)
tensor(6.9375, grad_fn=<NllLossBac